In [36]:
#from setup import *
##from agents import *
#from env import *
buyerStrategies = ['SAC']
sellerStrategies = ['TruthTeller', 'TruthTeller', 'TruthTeller', 'TruthTeller']
gamereinforcers, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed = '1001', len(buyerStrategies), len(sellerStrategies), 3, 1, 10000, 9, 42
disclosure = ['rnd', 'period', 'step','currentBid','currentAsk','buy','sell','price','sale']
gameData = [gamereinforcers, numBuyers, numSellers, numTokens, numRounds, numPeriods, numSteps, seed] 
buyers, sellers = generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure)
log = Log(gameData, buyerStrategies, sellerStrategies, disclosure)

#Hyperparameters
lr_pi           = 0.0005
lr_q            = 0.001
init_alpha      = 0.01
gamma           = 0.98
batch_size      = 32
buffer_limit    = 50000
tau             = 0.01 # for target network soft update
target_entropy  = -1.0 # for automated alpha update
lr_alpha        = 0.001  # for automated alpha update

rewards = []
for rnd in range(numRounds):
    roundData = roundSetup(*gameData)
    buyerValues, sellerCosts, demand, supply, prices, clearingPrice, clearingQuantity  = roundData[0:7]
    [buyerReservationPrices, sellerReservationPrices, buyerSurplus, sellerSurplus, totalSurplus, buyerSurplusFrac, sellerSurplusFrac] = roundData[7:]
    log.addRound([rnd] + roundData)
    resetRounds(buyers, sellers, buyerValues, sellerCosts)
    for period in range(numPeriods):
        resetPeriods(buyers, sellers)
        periodprofit = 0
        bids_ = []
        for step in range(numSteps):
            resetSteps(buyers, sellers)
            bids, asks = collectOffers(buyers, sellers)
            bids_.append(bids[0])
            currentAsk, currentAskIdx, currentBid, currentBidIdx = bestOffers(bids, asks)
            price, buy, sell = trade(buyers, sellers, currentAsk, currentAskIdx, currentBid, currentBidIdx)
            bprofit, sprofit = 0, 0
            if price > 0:
                buyers[currentBidIdx].transact(price)
                sellers[currentAskIdx].transact(price)
                bprofit = buyers[currentBidIdx].stepProfits
                sprofit = sellers[currentAskIdx].stepProfits
                periodprofit += bprofit
                print(bprofit, bids[0])
            log.addStep([rnd, period, step, bids, asks, currentBid, currentBidIdx, currentAsk, currentAskIdx, buy, sell, price, price>0, bprofit, sprofit])
            observe(buyers, sellers, log.disclose()) # observe new state
            updateStates(buyers, sellers)
            print(np.mean(bids))
        updatePolicy(buyers, sellers)
        rewards.append(periodprofit)
        rewards.append(log.getPeriod(rnd, period).tail(9)[['bprofit', 'currentBidIdx']].groupby('currentBidIdx').sum().bprofit.item())
        if period % (numPeriods/numPeriods) == 0:
            print('\n', np.mean(rewards[-100:]))
            #display(log.getPeriod(rnd, period).tail(100)[['sprofit', 'currentAskIdx']].groupby('currentAskIdx').sum())
            #display(log.getPeriod(rnd,period))

79.0 22.8
22.8
48.550000000000004 71.6
71.6
77.0 3.5
3.5
nan
nan
nan
nan
nan
nan

 204.55
73.85000000000001 33.1
33.1
77.8 13.1
13.1
70.55000000000001 17.3
17.3
nan
nan
nan
nan
nan
nan

 213.37500000000003
72.4 36.0
36.0
75.05 18.6
18.6
39.7 79.0
79.0
nan
nan
nan
nan
nan
nan

 204.63333333333335
67.25 46.3
46.3
59.45 49.8
49.8
37.800000000000004 82.8
82.8
nan
nan
nan
nan
nan
nan

 194.6
85.2 10.4
10.4
36.65 95.4
95.4
66.2 26.0
26.0
nan
nan
nan
nan
nan
nan

 193.29
70.45 39.9
39.9
62.900000000000006 42.9
42.9
51.30000000000001 55.8
55.8
nan
nan
nan
nan
nan
nan

 191.85
75.25 30.3
30.3
72.5 23.7
23.7
40.75 76.9
76.9
nan
nan
nan
nan
nan
nan

 191.37142857142857
56.00000000000001 68.8
68.8
74.4 19.9
19.9
43.7 71.0
71.0
nan
nan
nan
nan
nan
nan

 189.2125
50.400000000000006 80.0
80.0
72.65 23.4
23.4
77.0 2.6
2.6
nan
nan
nan
nan
nan
nan

 190.41666666666669
50.10000000000001 80.6
80.6
54.1 60.5
60.5
56.150000000000006 46.1
46.1
nan
nan
nan
nan
nan
nan

 187.41000000000003
75.65 29.5
29.5
67.8

KeyboardInterrupt: 

In [ ]:
env.log.getPeriod(0,5687)

In [ ]:
env.graphSales(0,7599)

In [35]:
class SAC(Trader):
    def __init__(self, gameData, disclosure, index, buyer, reinforcer):
        super().__init__(gameData, disclosure, index, buyer, reinforcer)
        self.learningRate = 0.0002
        self.gamma = 0.99
        self.numActions = 100
        self.depth = 9
        self.numStates = 8 + 6 * self.depth
        history = [-1] * self.numStates
        self.state = np.nan_to_num(np.array(history, dtype = np.float32), nan=-9)
        self.memory = ReplayBuffer()
        self.q1, self.q2, self.q1_target, self.q2_target = QNet(lr_q), QNet(lr_q), QNet(lr_q), QNet(lr_q)
        self.pi = PolicyNet(lr_pi, self.numStates)
        self.q1_target.load_state_dict(self.q1.state_dict())
        self.q2_target.load_state_dict(self.q2.state_dict())
        self.done = False
        self.truncated = False
        self.info = {}

    def observe(self):
        agentState = [self.periodStep, self.stepTokenValue, self.stepBid, self.stepAsk, self.stepTrades, self.stepProfits, self.periodTrades, self.periodProfits]
        history = self.df.iloc[-self.depth:][['currentBid', 'currentAsk', 'buy', 'sell', 'price', 'price']].values.reshape(-1,).tolist()
        if len(history) != (6 * self.depth):
            history = [-1] * (6 * self.depth)
        self.newState = np.nan_to_num(np.array(agentState + history, dtype = np.float32), nan=-9)
        if self.periodStep == self.numSteps:
            self.done = True
        memory.put((self.state, self.a.item(), self.stepProfits, self.newState, self.done)) 
        if memory.size()>1000:
            for i in range(20):
                mini_batch = memory.sample(batch_size)
                td_target = calc_target(self.pi, self.q1_target, self.q2_target, mini_batch)
                self.q1.train_net(td_target, mini_batch)
                self.q2.train_net(td_target, mini_batch)
                entropy = pi.train_net(self.q1, self.q2, mini_batch)
                self.q1.soft_update(self.q1_target)
                self.q2.soft_update(self.q2_target)
        self.state = self.newState
        
    def bid(self):
        self.stepBid = np.nan
        if self.stepTokenValue > 0:
            self.a, self.log_prob = self.pi(torch.from_numpy(self.state).float())
            frac = (self.a.item()+1)/2
            self.stepBid = np.round(frac * 100,1)
        return self.stepBid
        
    def ask(self):
        self.stepAsk = np.nan
        if self.stepTokenValue > 0:
            self.a, self.log_prob = self.pi(torch.from_numpy(self.state).float())
            frac = (self.a.item()+1)/2
            self.stepAsk = np.round(frac * 100,1)
        return self.stepAsk

In [24]:
import gym
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.distributions import Normal
import numpy as np
import collections, random

#Hyperparameters
lr_pi           = 0.0005
lr_q            = 0.001
init_alpha      = 0.01
gamma           = 0.98
batch_size      = 32
buffer_limit    = 50000
tau             = 0.01 # for target network soft update
target_entropy  = -1.0 # for automated alpha update
lr_alpha        = 0.001  # for automated alpha update

class ReplayBuffer():
    def __init__(self):
        self.buffer = collections.deque(maxlen=buffer_limit)

    def put(self, transition):
        self.buffer.append(transition)
    
    def sample(self, n):
        mini_batch = random.sample(self.buffer, n)
        s_lst, a_lst, r_lst, s_prime_lst, done_mask_lst = [], [], [], [], []

        for transition in mini_batch:
            s, a, r, s_prime, done = transition
            s_lst.append(s)
            a_lst.append([a])
            r_lst.append([r])
            s_prime_lst.append(s_prime)
            done_mask = 0.0 if done else 1.0 
            done_mask_lst.append([done_mask])
        
        return torch.tensor(s_lst, dreinforcer=torch.float), torch.tensor(a_lst, dreinforcer=torch.float), \
                torch.tensor(r_lst, dreinforcer=torch.float), torch.tensor(s_prime_lst, dreinforcer=torch.float), \
                torch.tensor(done_mask_lst, dreinforcer=torch.float)
    
    def size(self):
        return len(self.buffer)

class PolicyNet(nn.Module):
    def __init__(self, learning_rate, numStates):
        super(PolicyNet, self).__init__()
        self.fc1 = nn.Linear(numStates, 128)
        self.fc_mu = nn.Linear(128,1)
        self.fc_std  = nn.Linear(128,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)
        self.log_alpha = torch.tensor(np.log(init_alpha))
        self.log_alpha.requires_grad = True
        self.log_alpha_optimizer = optim.Adam([self.log_alpha], lr=lr_alpha)

    def forward(self, x):
        x = F.relu(self.fc1(x))
        mu = self.fc_mu(x)
        std = F.softplus(self.fc_std(x))
        dist = Normal(mu, std)
        action = dist.rsample()
        log_prob = dist.log_prob(action)
        real_action = torch.tanh(action)
        real_log_prob = log_prob - torch.log(1-torch.tanh(action).pow(2) + 1e-7)
        return real_action, real_log_prob

    def train_net(self, q1, q2, mini_batch):
        s, _, _, _, _ = mini_batch
        a, log_prob = self.forward(s)
        entropy = -self.log_alpha.exp() * log_prob
        q1_val, q2_val = q1(s,a), q2(s,a)
        q1_q2 = torch.cat([q1_val, q2_val], dim=1)
        min_q = torch.min(q1_q2, 1, keepdim=True)[0]
        loss = -min_q - entropy # for gradient ascent
        self.optimizer.zero_grad()
        loss.mean().backward()
        self.optimizer.step()
        self.log_alpha_optimizer.zero_grad()
        alpha_loss = -(self.log_alpha.exp() * (log_prob + target_entropy).detach()).mean()
        alpha_loss.backward()
        self.log_alpha_optimizer.step()

class QNet(nn.Module):
    def __init__(self, learning_rate):
        super(QNet, self).__init__()
        self.fc_s = nn.Linear(3, 64)
        self.fc_a = nn.Linear(1,64)
        self.fc_cat = nn.Linear(128,32)
        self.fc_out = nn.Linear(32,1)
        self.optimizer = optim.Adam(self.parameters(), lr=learning_rate)

    def forward(self, x, a):
        h1 = F.relu(self.fc_s(x))
        h2 = F.relu(self.fc_a(a))
        cat = torch.cat([h1,h2], dim=1)
        q = F.relu(self.fc_cat(cat))
        q = self.fc_out(q)
        return q

    def train_net(self, target, mini_batch):
        s, a, r, s_prime, done = mini_batch
        loss = F.smooth_l1_loss(self.forward(s, a) , target)
        self.optimizer.zero_grad()
        loss.mean().backward()
        self.optimizer.step()

    def soft_update(self, net_target):
        for param_target, param in zip(net_target.parameters(), self.parameters()):
            param_target.data.copy_(param_target.data * (1.0 - tau) + param.data * tau)

def calc_target(pi, q1, q2, mini_batch):
    s, a, r, s_prime, done = mini_batch

    with torch.no_grad():
        a_prime, log_prob= pi(s_prime)
        entropy = -pi.log_alpha.exp() * log_prob
        q1_val, q2_val = q1(s_prime,a_prime), q2(s_prime,a_prime)
        q1_q2 = torch.cat([q1_val, q2_val], dim=1)
        min_q = torch.min(q1_q2, 1, keepdim=True)[0]
        target = r + gamma * done * (min_q + entropy)
    return target
 

In [25]:
def generateAgents(gameData,buyerStrategies,sellerStrategies,disclosure):
    buyers, sellers = [], []
    for idx,i in enumerate(buyerStrategies):
        if i == 'TruthTeller':
            buyers.append(TruthTeller(gameData, disclosure, index=idx, buyer=1, reinforcer=0)) 
        if i == 'ZeroIntelligence':
            buyers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=1, reinforcer=0)) 
        if i == 'REINFORCE':
            buyers.append(REINFORCE(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'PPO':
            buyers.append(PPO(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'SAC':
            buyers.append(SAC(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 
        if i == 'DQN':
            buyers.append(DQN(gameData, disclosure, index=idx, buyer=1, reinforcer=1)) 

    for idx,i in enumerate(sellerStrategies):
        if i == 'TruthTeller':
            sellers.append(TruthTeller(gameData, disclosure, index=idx, buyer=0, reinforcer=0)) 
        if i == 'ZeroIntelligence':
            sellers.append(ZeroIntelligence(gameData, disclosure, index=idx, buyer=0, reinforcer=0)) 
        if i == 'REINFORCE':
            sellers.append(REINFORCE(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'PPO':
            sellers.append(PPO(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'SAC':
            sellers.append(SAC(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
        if i == 'DQN':
            sellers.append(DQN(gameData, disclosure, index=idx, buyer=0, reinforcer=1)) 
    return buyers, sellers